<a href="https://colab.research.google.com/github/danielmancovidales/Aplicaciones-en-finanzas/blob/main/Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [21]:
# Lectura de base de datos
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
df= pd.read_csv('/content/drive/MyDrive/analitica3/Financiera/df_final2.csv')
df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [23]:
df = df.dropna(subset=['ciudad'])


In [24]:
df.head(2)

,eventos,valor_pagado,ciudad,cancer,epoc,diabetes,hipertension,enf_cardiovascular,genero,edad,cantidad_enfermedades,clasifica_diagn
0,1,715463.04,1.0,0,0,0,0,0,1,27,0,2
1,1,691983.39,1.0,0,1,0,0,0,0,42,1,2


In [25]:
columnas = ['edad','epoc','diabetes','clasifica_diagn','ciudad','hipertension','cancer','enf_cardiovascular']

In [26]:
y = df['valor_pagado']
X = df[columnas]

In [27]:
# Si los valores de y son continuos, convertirlos a enteros
if y.dtype != 'int':
    y = y.round().astype(int)



In [28]:
#Comprobar tamaño de variables
print("Tamaño de X :", X.shape)
print("Tamaño de y :", y.shape)

Tamaño de X : (26285, 8)
Tamaño de y : (26285,)


In [29]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
# Verificar las dimensiones después de la división
print("Dimensiones de X_train:", X_train.shape)
print("Dimensiones de X_test:", X_test.shape)
print("Dimensiones de y_train:", y_train.shape)
print("Dimensiones de y_test:", y_test.shape)

Dimensiones de X_train: (21028, 8)
Dimensiones de X_test: (5257, 8)
Dimensiones de y_train: (21028,)
Dimensiones de y_test: (5257,)


In [31]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Ajustar el escalador solo en el conjunto de entrenamiento
X_train = scaler.fit_transform(X_train)

# Transformar el conjunto de prueba
X_test = scaler.transform(X_test)

In [32]:
# Verificar las dimensiones después del preprocesamiento
print("Dimensiones de X_train después del escalado:", X_train.shape)
print("Dimensiones de X_test después del escalado:", X_test.shape)

Dimensiones de X_train después del escalado: (21028, 8)
Dimensiones de X_test después del escalado: (5257, 8)


##Regresión Lineal

In [33]:
# Ajustar el modelo
model = LinearRegression()
model.fit(X_train, y_train)

# Predecir y evaluar el modelo
y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print('R2:',r2_score(y_test, y_pred))

MSE: 5914485824411.961
R2: 0.035348596356535866


##Regresión Ridge y Lasso

In [34]:
# Ridge Regression
ridge = Ridge(alpha=1.0)
ridge_scores = cross_val_score(ridge, X, y, cv=5, scoring='neg_mean_squared_error')
print("Ridge MSE:", -ridge_scores.mean())

# Lasso Regression
lasso = Lasso(alpha=0.1)
lasso_scores = cross_val_score(lasso, X, y, cv=5, scoring='neg_mean_squared_error')
print("Lasso MSE:", -lasso_scores.mean())
print('R2:',r2_score(y_test, y_pred))

Ridge MSE: 5857336061336.568
Lasso MSE: 5857340699454.616
R2: 0.035348596356535866


##Random Forest

In [35]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión con Random Forest
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

# Entrenar el modelo
rf_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_regressor.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)

MSE: 5312525069410.045
RMSE: 2304891.5526354043
MAE: 1562735.6629264466
R-squared: 0.13352826987170763


##XGBoost

In [36]:
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))

MSE: 5305282655550.235


In [37]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión con Decision Tree Regressor
dt_regressor = DecisionTreeRegressor()

# Entrenar el modelo
dt_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = dt_regressor.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)


MSE: 6023760256033.5625
RMSE: 2454334.992627038
MAE: 1614431.9787802154
R-squared: 0.017525959363978716


In [38]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos de entrada
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Crear el modelo de regresión con SGDRegressor
sgd_regressor = SGDRegressor()

# Entrenar el modelo
sgd_regressor.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = sgd_regressor.predict(X_test)

# Calcular las métricas de evaluación
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Imprimir las métricas de evaluación
print("MSE:", mse)
print("RMSE:", rmse)
print("MAE:", mae)
print("R-squared:", r2)


MSE: 5939106993084.153
RMSE: 2437028.3119168216
MAE: 1731980.7757127758
R-squared: 0.03133288888439323


# **Despliegue**

In [41]:
## se guarda el modelo random forest ya que entre todos los modelos aplicados es el mejor, con un R-squared: 0.13352826987170763

import pickle
nombreArchivo = 'modelo_final.sav'
pickle.dump(rf_regressor, open(nombreArchivo, 'wb'))

In [42]:
modeloCargado = pickle.load(open(nombreArchivo, 'rb'))
result = modeloCargado.score(X_test, y_test)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [43]:
import joblib
joblib.dump(rf_regressor, "rf_model.pkl")

['rf_model.pkl']

In [44]:
model = joblib.load('rf_model.pkl')
y_predict = model.predict(X_test)
print(y_predict)

[2053976.66254762 2097523.28972692 2049940.63414234 ... 2097523.28972692
 2097523.28972692 2137438.36632606]


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [45]:
X.columns

Index(['edad', 'epoc', 'diabetes', 'clasifica_diagn', 'ciudad', 'hipertension',
       'cancer', 'enf_cardiovascular'],
      dtype='object')

In [47]:
data_pred={
    'edad': 40,
    'epoc': 1,
    'diabetes':0,
    'clasifica_diagn':3,
    'ciudad': 2,
    'hipertension':0,
    'cancer':0,
    'enf_cardiovascular': 1

}

In [48]:
data = np.array(list(data_pred.values())).reshape(1, -1)
y_predict = model.predict(data)
print('El costo del seguro es: ',y_predict[0] )

El costo del seguro es:  4414128.105146275


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
